In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


等号之间不能有空格
D:\>tensorboard --logdir=D:\Tensorflow\logs 

In [2]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size


def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    x_image = tf.reshape(x,[-1,28,28,1])
    
keep_prob = tf.placeholder(tf.float32)


with tf.name_scope('conv_layer_1'):  
    #构建神经网络
    with tf.name_scope('weights'): 
        W_conv1 = weight_variable([5,5,1,10]) # 5*5的采样窗口，10个卷积核从1个平面抽取特征
    with tf.name_scope('biase'): 
        b_conv1 = bias_vairable([10]) #每个卷积核一个偏置值
    # 28*28*1 的图片卷积之后变为28*28*10
    with tf.name_scope('wx_plus_b'): 
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        
    with tf.name_scope('pool_layer_1'): 
    # 池化之后变为 14*14*10
        h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope('conv_layer_2'):   
    # 第二次卷积之后变为 14*14*64
    with tf.name_scope('weights'):
        W_conv2 = weight_variable([5,5,10,3])
    with tf.name_scope('biase'):
        b_conv2 = bias_vairable([3])
    with tf.name_scope('wx_plus_b'): 
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
    # 第二次池化之后变为 7*7*3
    with tf.name_scope('pool_layer_2'): 
        h_pool2 = max_pool_2x2(h_conv2)
        # 7*7*64的图像变成1维向量
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*3])

with tf.name_scope('dense_1'):   
    # 第一个全连接层
    with tf.name_scope('weights'):
        W_fc1 = weight_variable([7*7*3,30])
    with tf.name_scope('biase'):    
        b_fc1 = bias_vairable([30])
    with tf.name_scope('relu'): 
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    with tf.name_scope('drop'): 
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


# 第二个全连接层
with tf.name_scope('dense_2'):   
    # 第一个全连接层
    with tf.name_scope('weights'):
        W_fc2 = weight_variable([30,10])
    with tf.name_scope('biase'):  
        b_fc2 = bias_vairable([10])
    with tf.name_scope('wx_plus_b'): 
        logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
    with tf.name_scope('sigmoid'):   
        prediction = tf.nn.sigmoid(logits)
        prediction_2 = tf.nn.softmax(prediction)
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = (tf.equal(tf.argmax(prediction_2,1), tf.argmax(y,1)))
    with tf.name_scope('calculate_accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

merged = tf.summary.merge_all()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('temp/',sess.graph)
    for epoch in range(20):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step], feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter: " + str(epoch) + ", acc: " + str(acc))

Iter: 0, acc: 0.935
Iter: 1, acc: 0.9533
Iter: 2, acc: 0.9624
Iter: 3, acc: 0.9661
Iter: 4, acc: 0.971
Iter: 5, acc: 0.9724
Iter: 6, acc: 0.9748
Iter: 7, acc: 0.9754
Iter: 8, acc: 0.9771
Iter: 9, acc: 0.9789
Iter: 10, acc: 0.9796
Iter: 11, acc: 0.9792
Iter: 12, acc: 0.9816
Iter: 13, acc: 0.9818
Iter: 14, acc: 0.9801
Iter: 15, acc: 0.9815
Iter: 16, acc: 0.9843
Iter: 17, acc: 0.9828
Iter: 18, acc: 0.9835
Iter: 19, acc: 0.9842
